### 3-1. 네이버 책 검색 API 호출하기 (필수)
https://developers.naver.com/docs/serviceapi/search/book/book.md#%EC%B1%85

* 검색어(query)는  함수의 인자로 받아서 동적으로 처리 되어야 합니다. 
* 최대한 코드가 중복되지 않도록 공통으로 처리해야 하는 부분은 함수로 작성해서 재사용 하는 방식으로 코드를 작성해 주세요.

def search_books(query):   
    # query string 문자열을 dict 선언
    payload = {
    'query':  query, #'파이썬',
    'display': 50,
    'sort': 'sim'
    }

   url = 'https://openapi.naver.com/v1/search/book.json'
   
   *requests get(url, params, headers) 요청 
      res = requests.get(url, params=payload, headers=headers)
   *json() 함수로 응답 결과 가져오기
      items_data = res.json()['items'] 

1. 질문 :  검색어로  찾은  책 목록을 json 파일로 저장하기
   data/books.json 파일로 저장해 주세요.

2. books.json 파일을 Pandas DataFrame로 저장하기

3. 질문 :  검색어로  찾은  책 목록 출력하기


In [56]:
#검색어로 찾은 책 목록을 json 파일에 저장하기

import requests
from pprint import pprint
import os
from dotenv import load_dotenv
import pandas as pd


# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}


def search_books(query):   
    # query string 문자열을 dict 선언
    payload = {
    'query':  query, #'파이썬',
    'display': 50,
    'sort': 'sim'
    }

    book_url = 'https://openapi.naver.com/v1/search/book.json'

    # requests get(url, params, headers) 요청 
    res = requests.get(book_url, params=payload, headers=headers)

    # json() 함수로 응답 결과 가져오기
    items_data = res.json()['items']
    #pprint(items_data[:2])

    # 'data/books.json' 파일 생성하기
    items_list = list()
    item_list = []
    for item in items_data :
        item_list.append(item['title'])
        item_list.append(item['link'])
        item_list.append(item['image'])
        item_list.append(item['author'])
        item_list.append(item['discount'])
        item_list.append(item['publisher'])
        item_list.append(item['pubdate'])
        item_list.append(item['isbn'])
        item_list.append(item['description'])
        

        items_list.append(item_list)
        item_list = []

    with open('../data/books.json','w',encoding="utf-8")as file:
        for items in items_list:
            for item in items:
                item = item + '\n'
                file.write(item)
            file.write('-'*150+'\n')
    
    return items_data
    

In [ ]:
items_data = search_books('파이썬')
print(type(items_data))

books_df = pd.DataFrame(columns = ['title','link','image','author','discount','publisher','pubdate','isbn','description'])

for book_info in items_data:
    df_new_row = pd.DataFrame.from_records([book_info])
    books_df = pd.concat([books_df, df_new_row])

books_df.reset_index(drop=True,inplace=True)

In [ ]:
books_df.head()

##### 2. Json 파일을 받아서 dataframe 만들기(description이 없음)

In [66]:
# books.json 파일을 읽어서 dataframe으로 만들기
import pandas as pd

line=''
book=[]
books_list=[]
book_columns = ['title','link','image','author','discount','publisher','pubdate','isbn']

# data/books.json 파일 읽어오기
with open('../data/books.json','r',encoding="utf-8")as file:
    items = file.read()

#book에 대한 dict로 저장하기
for item in items :
    if item == '\n':
        if line == '-'*150:
            line = ''
            books_list.append(dict(zip(book_columns,book)))
            book=list()
        else:
            book.append(line)
            line=''
    else :
        if item=='^':
            item = ','
        line = line + str(item)


book_df = pd.DataFrame(columns = book_columns)

for book_info in books_list:
    df_new_row = pd.DataFrame.from_records([book_info])
    book_df = pd.concat([book_df, df_new_row])




In [67]:
book_df.reset_index(drop=True,inplace=True)
book_df.head()

,title,link,image,author,discount,publisher,pubdate,isbn
0,처음이야? 파이썬 기초 (동영상 강의로 배우는 292개 코드 따라하기(핵심노트+오픈...,https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_550256...,"윤영빈,오환,이용희",18000,영진닷컴,20250620,9788931478006
1,모두의 인공지능 with 파이썬 (누구나 쉽게 시작하는 AI 기초 프로그래밍),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_558136...,이영호,27000,길벗,20250725,9791140714995
2,혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_325076...,윤인성,19800,한빛미디어,20220601,9791162245651
3,Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_403540...,박응용,19800,이지스퍼블리싱,20230615,9791163034735
4,파이썬,https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_324609...,홍의경,30400,생능출판,20220309,9788970506784
